# XML Parser Test

In [1]:
import re
from collections import Counter
from lxml import etree

tree = etree.parse('../datasets/Kranjska-xml/Corpus-Kranjska.xml')
tree.xinclude()
root = tree.getroot()

xmlns='http://www.w3.org/XML/1998/namespace'

This is a small test to see if we can traverse the DOM. We count the number of meetings in year 1895.

DOC: https://lxml.de/tutorial.html#parsing-from-strings-and-files

In [2]:
# Function to strip namespace from tag
def strip_namespace(tag):
    return tag.split('}')[-1] if '}' in tag else tag

# Travers the DOM tree recursively
def traverse_tree(element, meeting_dict, update_id):
    import re
    xmlns='http://www.w3.org/XML/1998/namespace'
    skip_meeting = False

    # Regular expression pattern to match the year (four digits)
    pattern = r'\b\d{4}\b'
    
    # TODO - save elements we want to save else continue the traverse
    if strip_namespace(element.tag) == 'meeting':
            
            if element.text is not None: # sanity check
                if element.attrib.get(f'{{{xmlns}}}lang') == 'de':
                    update_id[0] +=1
                    
                    # extract year
                    match = re.search(pattern, element.text)

                    if match:
                        year = match.group()
                        
                        # Initialize new meeting
                        meeting_dict['id'].append(update_id[0])
                        meeting_dict['year'].append(year)
                        meeting_dict['title'].append(element.text)
                        meeting_dict['speakers'].append([])

                        skip_meeting = False
                    else: 
                        skip_meeting = True
                        update_id[1] += 1


    if strip_namespace(element.tag) == 'note' and not skip_meeting:
         
         if element.text is not None: # sanity check
              if element.attrib.get(f'type') == 'speaker':
                   
                   # update list of spekers
                   meeting_dict['speakers'][-1].append(element.text)
                   
    for child in element:
        traverse_tree(child, meeting_dict, update_id)


In [3]:
update_id = [0, 0]

meeting_dict = {
    'id': [],
    'title': [],
    'year': [],
    'speakers': []
}

traverse_tree(root, meeting_dict, update_id)  

In [7]:
print(meeting_dict)
print('n-meetings: ', update_id[0], 'n-skipped: ', update_id[1])

{'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222,

## Sanity check and Post processing

We check that we have one row per meeting

In [12]:
print('Numb of Meetings', 
      len(meeting_dict['id']), 
      len(meeting_dict['year']),
      len(meeting_dict['title']), 
      len(meeting_dict['speakers'])
)

Numb of Meetings 667 667 667 667


In [13]:
#meeting_dict['id']

sanity check years

In [14]:
#set(meeting_dict['year'])

Count how often a speaker speaks

In [15]:
#meeting_dict['title']

In [16]:
meeting_dict['speakers'][1]


from collections import Counter

speakers_counter = []
for meeting in meeting_dict['speakers']:
    speakers_counter.append(dict(Counter(meeting)))

meeting_dict['speakers'] = speakers_counter

display(len(meeting_dict['speakers'][1]))


30

In [17]:
import json
import os

data = meeting_dict

# Define the filename for the JSON file
fname = "../cache/out.json"
dir = os.path.dirname(fname)

# Check directory
if not os.path.exists(dir):
    os.makedirs(dir)

# Write the dictionary to a JSON file
with open(fname, "w") as json_file:
    json.dump(data, json_file)

print("Data saved to", fname)

Data saved to ../cahe/out.json
